In [1]:
# Imports
import gym
import numpy as np
import time
from gym.envs.registration import register
from IPython.display import clear_output

In [2]:
env_name = "FrozenLakeNoSlip-v0"
custom_map = ["SFFF", 
              "FHFH", 
              "FFFH", 
              "HFFG"]

try:
    register(
        id='FrozenLakeNoSlip-v0',
        entry_point='gym.envs.toy_text:FrozenLakeEnv',
        kwargs={'is_slippery':False},
        max_episode_steps=100,
        reward_threshold=0.78,
    )
except:
    pass

env = gym.make(env_name, desc = custom_map)
print(env.observation_space)
print(env.action_space)

Discrete(16)
Discrete(4)


In [3]:
import pygame, sys

SCREEN_WIDTH = 480
SCREEN_HEIGHT = 480

NUM_BLOCKS_WIDE = 4
NUM_BLOCKS_HIGH = 4
BLOCK_HEIGHT = round(SCREEN_HEIGHT/NUM_BLOCKS_HIGH)
BLOCK_WIDTH = round(SCREEN_WIDTH/NUM_BLOCKS_WIDE)

BLACK = (0, 0, 0)
START = (153, 184, 152) # color is white
GOAL = (0,201,87) # color is green
HOLE = (139,105,105) # color is brown
FROZEN = (24,116,205) # color is blue
PINK= (255,20,147) # color is pink

TITLE = "4x4 Forzen Lake"

def return_coordinates(state):
    count = 0
    matrix = []
    
    for _ in range(len(custom_map)):
        row = []
        for _ in range(len(custom_map[0])):
            row.append(count)
            count+=1
        matrix.append(row)
    
    for i in range(len(custom_map)):
        for j in range(len(custom_map[0])):
            if matrix[i][j] == state:
                pos = (i, j)
    
    return pos

pygame 2.1.2 (SDL 2.0.18, Python 3.9.2)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
def get_tile_color(tile):
    if tile == "S":
        color = START
    elif tile == "G":
        color = GOAL
    elif tile == "H":
        color = HOLE
    elif tile == "F":
        color = FROZEN
    else:
        color = PINK
    return color

In [5]:
def draw_map(surface, map_tiles):
    for j, tile in enumerate(map_tiles):
        for i, tile_contents in enumerate(tile):
            myrect = pygame.Rect(i * BLOCK_WIDTH, j * BLOCK_HEIGHT, BLOCK_WIDTH, BLOCK_HEIGHT)
            pygame.draw.rect(surface, get_tile_color(tile_contents), myrect)

def draw_grid(surface):
    for i in range(NUM_BLOCKS_WIDE):
        new_height = round(i * BLOCK_HEIGHT)
        new_width = round(i * BLOCK_WIDTH)
        pygame.draw.line(surface, BLACK, (0, new_height), (SCREEN_WIDTH, new_height), 2)
        pygame.draw.line(surface, BLACK, (new_width, 0), (new_width, SCREEN_HEIGHT), 2)


In [6]:
def initialize_game():
    pygame.init()
    surface = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption(TITLE)
    surface.fill(BLACK)
    return surface

def read_map(state_pos):
    
    world_map = custom_map      
    world_map = []
    for row in custom_map:
        world_map.append(list(row))
        
    world_map[state_pos[0]][state_pos[1]] = "X"
    
    new_map = []
    for row in world_map:
        string = ""
        for ele in row:
            string += ele
        new_map.append(string)
                
    return new_map

In [7]:
import pickle
# with open("pygame_custom_qtable.pkl", "wb") as f:
    # pickle.dump(agent.q_table, f)
with open("pygame_custom_qtable.pkl", "rb") as f:
    loaded_table = pickle.load(f)
loaded_table

array([[4.40803515e-05, 4.39107855e-03, 4.06461874e-05, 4.06669095e-05],
       [4.03967506e-05, 4.00910533e-05, 2.66905698e-05, 1.43512302e-05],
       [1.30978208e-05, 5.70795992e-05, 7.48783212e-05, 9.05043092e-05],
       [5.65749520e-05, 7.38228703e-06, 1.14858796e-05, 2.89229161e-05],
       [4.18790101e-05, 1.84255618e-02, 1.93517396e-05, 4.10007553e-05],
       [3.06525882e-05, 2.14175091e-05, 2.06074701e-05, 7.10915507e-05],
       [1.10024654e-05, 8.42097084e-04, 2.94411863e-05, 4.61070559e-05],
       [9.66623140e-05, 3.15145754e-05, 6.40110212e-05, 7.08949743e-05],
       [2.98697831e-05, 4.07063165e-05, 6.55484989e-02, 4.10526548e-05],
       [6.89133183e-05, 4.30267057e-05, 1.91813373e-01, 6.90786778e-05],
       [1.46280426e-04, 4.45897417e-01, 7.40361170e-05, 6.92308294e-05],
       [2.96752509e-05, 4.20845884e-05, 8.97933885e-05, 1.70667459e-05],
       [8.17326033e-05, 5.21196582e-05, 3.40291996e-05, 4.76432688e-05],
       [2.29719747e-05, 4.73916143e-05, 8.91272618e

In [8]:
def game_loop(surface):
    
    total_reward = 0
    rounds = 5
    num_eps = 100

    for i in range(1, rounds + 1):

        for _ in range(1, num_eps + 1):

            state = env.reset()
            done = False
            while not done:

                for event in pygame.event.get():
                    if event.type == pygame.QUIT:
                        pygame.quit()
                        sys.exit()
                    if event.type == pygame.KEYDOWN:
                        if event.key == pygame.K_ESCAPE:
                            pygame.quit()
                            sys.exit()
                action = np.argmax(loaded_table[state, :])
                new_state, reward, done, _ = env.step(action)
                state = new_state
                total_reward += reward

                print("Round", i)
                print("s:", state, "a:", action)
                env.render()
                time.sleep(0.05)
                clear_output(wait=True)

                draw_map(surface, read_map(return_coordinates(state)))
                draw_grid(surface)
                pygame.display.update()

        total_reward = 0

In [10]:
world_map = custom_map
surface = initialize_game()
game_loop(surface)

SystemExit: 